# CompLabNGS2025 Final Project

## Getting the data
Visit the [Gene Expression Omnibus (GEO) website](https://www.ncbi.nlm.nih.gov/geo/) and search for the accession number you were given. Review the information on the dataset and explore the publication that generated the data.

Write a short but inclusive description of the dataset you were given. Include name of the dataset, the publication, authors, number and condition of samples and any other relevant information to allow others to understand the dataset.

After completing the entire analysis, create a conda environment with all the necessary packages and their versions to run the analysis. In the cell below, write the command you used to create the environment and the command to activate it. The name of the environment should be `<first name>_<last name>`. You can use ```bash conda list``` to list all the packages and their versions in the environment.

Modify the example command below to include the packages you used in your analysis.

In [ ]:
!conda create -n hadas_volk \
    python=3.7 \
    pandas=1.0.3 \
    numpy=1.18.1 \
    matplotlib=3.1.3 \
    scikit-learn=0.22.1 \
    pydeseq2=1.26.0
!conda activate hadas_volk

## Getting the raw fastq files
At the bottom of the GEO page, you will find a link to the `SRA Run Selector`. Click on the link. 

Explain what is the SRA and SRR in gerneal and how it is used to store sequencing data.


Use `prefetch + fasterq-dump` command from the `SRA Toolkit` to download the fastq files. Write the command you used to download the fastq files. [SRA Tools Conda installation](https://anaconda.org/bioconda/sra-tools)

## Quality Control
Below include all the commands, figures and output you used and generated to perform quality control on the fastq files. You can use `fastqc`, `trimmomatic`, `multiqc` and any other tool you find necessary to give a comprehensive QC and filtering of the data.

Write a brief description of the quality control process and explain the figures and tables you generated.

## Alignment and DGE

Similar to what we did in class and in the assignemnts, you will now run through the steps of mapping and conducting DGE. Include below an elaborate report of your analysis, include study desgin, example count and normalized count matrices, draw summary satistics and every other information you think is relevent to report your findings (if any)

Running `STAR` on a laptop or PC is almost impossible we will use one of the more recent pseudo alighners, ['kallisto`](https://pachterlab.github.io/kallisto/manual), which will allow us to quantify transcript level information in a more efficent manner.

1. Download the index files from kallisto's GitHub
```bash
wget https://github.com/pachterlab/kallisto-transcriptome-indices/releases/download/v1/mouse_index_standard.tar.xz
tar -xf mouse_index_standard.tar.xz
```
2. Install and run `kallisto`
```bash
conda install -c bioconda -c conda-forge kallisto==0.50.1
kallisto quant -i index.idx -o output -t 16 sample.fastq 
```

In the `output` directory you should have `abundance.tsv`. View the file and make sure you understand it. Produce quantification to all your samples.
We now need to sum up transcript level counts to gene level counts before proceeding to `pyDESEQ`


```python
import pandas as pd

# Load transcript to gene mapping file
t2g_file = 't2g.txt'
t2g_df = pd.read_csv(t2g_file, sep='\t', usecols=[0, 2], header=None, names=['transcript_id', 'gene_id'])

abundance_file = 'output/abundance.tsv'
abundance_df = pd.read_csv(abundance_file, sep='\t')

# Merge transcript abundance with gene mapping
merged_df = pd.merge(abundance_df, t2g_df, left_on='target_id', right_on='transcript_id', how='inner')

# Sum abundances at the gene level
gene_abundance_df = merged_df.groupby('gene_id')['tpm'].sum().reset_index()

# Save gene-level abundance to a new file
print(gene_abundance_df)
gene_abundance_df.to_csv('gene_abundance.csv', index=False)
```

Do you understand the code? Include `print` statments and rerun until you understnad what's going on fully

Do this operation to each of your sample's abundance file. Using python or a different tool merge all samples to a single file.

You should now be able to take the resulting `all_samples_gene_abundance.csv` and use it in `pyDeSeq2` as we did in class

## Gene Set Enrichment Analysis

Explain what GSEA is and what insights can we draw from it.

Execute a GSEA analysis using [pyGSEA](https://gseapy.readthedocs.io/en/latest/introduction.html) and summarize your results below.

## Notes

This excerise is meant to 'through' in the water and might prove challanging. Attempt to work in groups to overcome challanges. 

If you get stuck try to post an issue, but give as much background and explanation of your problem as possible so I could help

### Good luck!